In [2]:
# --- ETL Transform Step ---
# Apply at least 4 meaningful transformations to both datasets, including categorization by modifying existing columns

import pandas as pd
from tabulate import tabulate
import os
import numpy as np

# Load extracted data
raw_df = pd.read_csv('data/raw_data.csv')
incremental_df = pd.read_csv('data/incremental_data.csv')

# Fix missing values in both dataframes before transformation
for df in [raw_df, incremental_df]:
    for col in df.columns:
        if df[col].dtype == 'O':  # Object type (string)
            df[col] = df[col].fillna('Unknown')
        elif pd.api.types.is_numeric_dtype(df[col]):
            mean_val = df[col].mean()
            if pd.notnull(mean_val):
                mean_val = float(f'{mean_val:.1g}')
            df[col] = df[col].fillna(mean_val)  # Fill numeric with rounded mean (1 sig fig)
        elif pd.api.types.is_datetime64_any_dtype(df[col]):
            df[col] = df[col].fillna(pd.Timestamp('1970-01-01'))
        else:
            df[col] = df[col].fillna('Unknown')

def reorder_columns(df):
    cols = list(df.columns)
    # Move 'product_category' after 'product' if present
    if 'product_category' in cols and 'product' in cols:
        cols.remove('product_category')
        prod_idx = cols.index('product')
        cols.insert(prod_idx + 1, 'product_category')
    # Move 'total_price' after 'unit_price'
    if 'total_price' in cols and 'unit_price' in cols:
        cols.remove('total_price')
        unit_idx = cols.index('unit_price')
        cols.insert(unit_idx + 1, 'total_price')
    return df[cols]

def categorize_product(product):
    # Example categorization logic, adjust as needed for your data
    if isinstance(product, str):
        p = product.lower()
        if any(x in p for x in ['shirt', 'pants', 'dress', 'clothes', 'apparel']):
            return 'Apparel'
        elif any(x in p for x in ['phone', 'laptop', 'tablet', 'electronics']):
            return 'Electronics'
        elif any(x in p for x in ['book', 'magazine', 'novel']):
            return 'Books'
        elif any(x in p for x in ['food', 'snack', 'beverage', 'drink']):
            return 'Food & Beverage'
        else:
            return 'Other'
    return 'Other'

def transform(df):
    # 1. Remove duplicate rows
    df = df.drop_duplicates().copy()
    
    # 2. Handle missing values for all columns
    for col in df.columns:
        if df[col].dtype == 'O':  # Object type (string)
            df.loc[:, col] = df[col].fillna('Unknown')
        elif pd.api.types.is_numeric_dtype(df[col]):
            mean_val = df[col].mean()
            if pd.notnull(mean_val):
                mean_val = float(f'{mean_val:.1g}')
            df.loc[:, col] = df[col].fillna(mean_val)  # Fill numeric with rounded mean (1 sig fig)
        elif pd.api.types.is_datetime64_any_dtype(df[col]):
            df.loc[:, col] = df[col].fillna(pd.Timestamp('1970-01-01'))
        else:
            df.loc[:, col] = df[col].fillna('Unknown')
    
    # 3. Convert date columns to datetime (example: 'Date' or 'date')
    for col in df.columns:
        if 'date' in col.lower():
            df.loc[:, col] = pd.to_datetime(df[col], errors='coerce')
            df.loc[:, col] = df[col].fillna(pd.Timestamp('1970-01-01'))
    
    # 4. Categorize 'product' into broader groups (add 'product_category' column)
    if 'product' in df.columns:
        df.loc[:, 'product_category'] = df['product'].apply(categorize_product)
    
    # 5. Remove time from 'order_date' if present
    if 'order_date' in df.columns:
        df.loc[:, 'order_date'] = pd.to_datetime(df['order_date'], errors='coerce').dt.strftime('%Y-%m-%d')
    
    # 6. Enrichment: Add total_price = quantity * unit_price
    if 'quantity' in df.columns and 'unit_price' in df.columns:
        df.loc[:, 'total_price'] = df['quantity'] * df['unit_price']
    
    # 7. Ensure quantity, unit_price, and total_price are whole numbers (integers)
    for col in ['quantity', 'unit_price', 'total_price']:
        if col in df.columns:
            df.loc[:, col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
    
    # 8. Reorder columns for output and CSV
    df = reorder_columns(df)
    
    return df

# Apply transformations
transformed_full = transform(raw_df)
transformed_incremental = transform(incremental_df)

# Save transformed files
os.makedirs('transformed', exist_ok=True)
transformed_full.to_csv('transformed/transformed_full.csv', index=False)
transformed_incremental.to_csv('transformed/transformed_incremental.csv', index=False)
print("\nTransformed files saved to 'transformed/transformed_full.csv' and 'transformed/transformed_incremental.csv'")

# Display only the head of the final outputs, with 'product_category' after 'product' and 'total_price' after 'unit_price'
def print_head_with_category(df, name):
    df = reorder_columns(df)
    print(f"\n=== Head of {name} ===")
    print(tabulate(df.head(), headers='keys', tablefmt='grid'))

print_head_with_category(transformed_full, 'transformed_full.csv')
print_head_with_category(transformed_incremental, 'transformed_incremental.csv')

# Show summary of transformations for verification
def transformation_summary(df, name):
    summary = [
        ['File', name],
        ['Rows', len(df)],
        ['Duplicate Rows', df.duplicated().sum()],
        ['Any Missing Values', df.isnull().any().any()],
    ]
    if 'product_category' in df.columns:
        summary.append(['Product Category Unique Values', ', '.join(df['product_category'].unique().astype(str))])
    print(tabulate(summary, tablefmt='grid'))

print("\n=== Transformation Summary: transformed_full.csv ===")
transformation_summary(transformed_full, 'transformed_full.csv')
print("\n=== Transformation Summary: transformed_incremental.csv ===")
transformation_summary(transformed_incremental, 'transformed_incremental.csv')


Transformed files saved to 'transformed/transformed_full.csv' and 'transformed/transformed_incremental.csv'

=== Head of transformed_full.csv ===
+----+------------+-----------------+-----------+--------------------+------------+--------------+---------------+--------------+----------+
|    |   order_id | customer_name   | product   | product_category   |   quantity |   unit_price |   total_price | order_date   | region   |
+====+============+=================+===========+====================+============+==============+===============+==============+==========+
|  0 |          1 | Diana           | Tablet    | Electronics        |          2 |          500 |          1000 | 2024-01-20   | South    |
+----+------------+-----------------+-----------+--------------------+------------+--------------+---------------+--------------+----------+
|  1 |          2 | Eve             | Laptop    | Electronics        |          2 |          500 |          1000 | 2024-04-29   | North    |
+----+-

C:\Users\jpcha\AppData\Local\Temp\ipykernel_12428\4047370646.py:80: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[:, col] = df[col].fillna(pd.Timestamp('1970-01-01'))
C:\Users\jpcha\AppData\Local\Temp\ipykernel_12428\4047370646.py:80: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[:, col] = df[col].fillna(pd.Timestamp('1970-01-01'))
